# Chapter 6: From MoCap Data to NeuroMSK Models and Machine Learning

**This notebook is part of Chapter 6.**

Motion capture data is powerful, but sometimes not enough. If we want to understand how joints, muscles, ligaments, and even the nervous system produce movement, or how they adapt in disease and after surgery, we need more than raw kinematics. This is where neuromusculoskeletal (NeuroMSK) models come in.

These models combine anatomical structures (bones, joints, muscles, ligaments) with physiological parameters, often derived from cadaver studies, to simulate movement. Using your MoCap data, the models apply:

- **Inverse dynamics**: starting from observed motion and external forces, we estimate joint moments and muscle forces that could have produced them.  
- **Forward dynamics**: starting from muscle activations, we simulate the resulting movement and compare it to reality.

**Ebook:** *A Hands-On Guide to Biomechanics Data Analysis with Python and AI*  
**Author:** Dr. Hossein Mokhtarzadeh  
**Powered by:** PoseIQ™

This notebook loads sample biomechanics data and shows how to bring it into Python.  
*In Colab:* go to **Runtime → Run all**.


In [ ]:
# (Optional) If Bokeh is missing in your runtime, uncomment the next line.
# !pip -q install bokeh

In [ ]:
# --- Interactive NeuroMSK Knee Model: ALL DEGREES IN IK + synced with Figure 4 ---
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot, column, row
from bokeh.models import ColumnDataSource, Slider, Button, CustomJS, Span, Label

output_notebook()

# ---------------- Angle & kinematics ----------------
t = np.linspace(0, 2*np.pi, 200)

# θ(deg): -90 (flexion) → +5 (hyperextension)
s = 0.5*(1+np.sin(t))               # 0..1
theta_deg = -90 + 95*s              # degrees
theta = np.radians(theta_deg)       # radians for trig/torque

dt = t[1]-t[0]
omega  = np.gradient(theta, dt)
alpha  = np.gradient(omega, dt)
torque = alpha                      # required torque with I=1

# ---------------- Smooth angle-based activations ----------------
u = (theta_deg - (-90.0)) / 95.0
act_flexor   = np.clip(0.5*(1 + np.cos(np.pi * u)), 0.0, 1.0)   # high in flexion
act_extensor = np.clip(0.5*(1 - np.cos(np.pi * u)), 0.0, 1.0)   # high near extension

Fmax_flexor, Fmax_extensor = 1000.0, 1000.0
ma_flexor, ma_extensor = 0.04, 0.04

F_flexor   = act_flexor   * Fmax_flexor
F_extensor = act_extensor * Fmax_extensor

# ✅ Torque signs: flexion (−), extension (+)
T_flexor   = -F_flexor   * ma_flexor
T_extensor =  F_extensor * ma_extensor
T_net      =  T_flexor + T_extensor

# ---------------- DataSources ----------------
src_ik     = ColumnDataSource(data=dict(t=t, theta_deg=theta_deg))
src_marker = ColumnDataSource(data=dict(x=[t[0]], y=[theta_deg[0]]))  # green IK dot
src_forces = ColumnDataSource(data=dict(t=t, F_flexor=F_flexor, F_extensor=F_extensor))
src_torque = ColumnDataSource(data=dict(t=t, torque=torque, T_net=T_net))

# ---------------- Plots ----------------
p1 = figure(width=400, height=300, title="IK: Joint Angle (deg)", y_axis_label="Angle (deg)")
p1.line("t", "theta_deg", source=src_ik, line_width=2, color="black", legend_label="θ (deg)")
p1.scatter("x", "y", source=src_marker, size=8, color="green")  # shows current angle on IK curve

p2 = figure(width=400, height=300, title="ID: Torque  (− = flexion, + = extension)")
p2.line("t", "torque", source=src_torque, line_width=2, color="black", legend_label="Required")
p2.line("t", "T_net",  source=src_torque, line_width=2, color="orange", legend_label="From Muscles")

p3 = figure(width=400, height=300, title="Muscle Forces")
p3.line("t", "F_flexor",   source=src_forces, line_width=2, color="red",  legend_label="Flexor (hamstrings)")
p3.line("t", "F_extensor", source=src_forces, line_width=2, color="blue", legend_label="Extensor (quads)")

# Figure 4: Knee + muscles
p4 = figure(width=400, height=300, title="Figure 4: Knee Joint + Muscles",
            x_range=(-0.5,0.5), y_range=(-0.5,0.5))
i0 = 0
x_thigh = [-0.4, 0.0]; y_thigh = [0.0, 0.0]
x_shank = [0.0, 0.4*np.cos(theta[i0])]
y_shank = [0.0, 0.4*np.sin(theta[i0])]
x_flex  = [0.0, x_shank[1]]; y_flex = [-0.2, y_shank[1]]
x_ext   = [0.0, x_shank[1]]; y_ext  = [ 0.2, y_shank[1]]

src_joint = ColumnDataSource(data=dict(
    x_thigh=x_thigh, y_thigh=y_thigh,
    x_shank=x_shank, y_shank=y_shank,
    x_flex=x_flex,   y_flex=y_flex,
    x_ext=x_ext,     y_ext=y_ext
))

thigh_glyph = p4.line("x_thigh", "y_thigh", source=src_joint, line_width=6, color="brown", legend_label="Thigh (fixed)")
shank_glyph = p4.line("x_shank", "y_shank", source=src_joint, line_width=6, color="black", legend_label="Shank (moving)")
flex_glyph  = p4.line("x_flex",  "y_flex",  source=src_joint, line_width=2, color="red",  legend_label="Flexor (hamstrings)")
ext_glyph   = p4.line("x_ext",   "y_ext",   source=src_joint, line_width=2, color="blue", legend_label="Extensor (quads)")
p4.scatter([0],[0], size=12, color="gray")  # knee joint center
p4.legend.location = "top_left"; p4.legend.background_fill_alpha = 0.5; p4.legend.label_text_font_size = "9pt"

angle_label = Label(x=-0.45, y=0.45,
                    text=f"Angle: {theta_deg[i0]:.1f}°",
                    text_color="green", text_font_size="10pt",
                    background_fill_color="white", background_fill_alpha=0.7)
p4.add_layout(angle_label)

# ---------------- Vertical time cursor ----------------
cursor1 = Span(location=t[0], dimension='height', line_color='green', line_width=2)
cursor2 = Span(location=t[0], dimension='height', line_color='green', line_width=2)
cursor3 = Span(location=t[0], dimension='height', line_color='green', line_width=2)
p1.add_layout(cursor1); p2.add_layout(cursor2); p3.add_layout(cursor3)

# ---------------- Controls ----------------
sFmaxF = Slider(start=100, end=2000, value=Fmax_flexor,   step=50, title="Flexor Max Force")
sFmaxE = Slider(start=100, end=2000, value=Fmax_extensor, step=50, title="Extensor Max Force")
sTime  = Slider(start=0, end=len(t)-1, value=0, step=1, title="Time Frame")
btnPlay = Button(label="▶ Play", width=60)

# ---------------- JS Callback ----------------
callback = CustomJS(args=dict(
    src_ik=src_ik, src_marker=src_marker,
    src_forces=src_forces, src_torque=src_torque, src_joint=src_joint,
    t=t, theta=theta, theta_deg=theta_deg,
    act_flexor=act_flexor, act_extensor=act_extensor,
    torque=torque, ma_flexor=ma_flexor, ma_extensor=ma_extensor,
    sFmaxF=sFmaxF, sFmaxE=sFmaxE, sTime=sTime,
    flex_line=flex_glyph, ext_line=ext_glyph,
    cursor1=cursor1, cursor2=cursor2, cursor3=cursor3, angle_label=angle_label
), code="""
    const FmaxF = sFmaxF.value;
    const FmaxE = sFmaxE.value;

    const F_flexor   = act_flexor.map(a => a * FmaxF);
    const F_extensor = act_extensor.map(a => a * FmaxE);

    // Torques: flexion (−), extension (+)
    const T_flexor   = F_flexor.map(f => -f * ma_flexor);
    const T_extensor = F_extensor.map(f =>  f * ma_extensor);
    const T_net = T_flexor.map((tf,i) => tf + T_extensor[i]);

    src_forces.data = {t: t, F_flexor: F_flexor, F_extensor: F_extensor};
    src_torque.data = {t: t, torque: torque, T_net: T_net};

    // Update joint pose
    const i = sTime.value;
    const ang = theta[i];
    const xb = [0, 0.4*Math.cos(ang)];
    const yb = [0, 0.4*Math.sin(ang)];
    const xf = [0, xb[1]];
    const yf = [-0.2, yb[1]];
    const xe = [0, xb[1]];
    const ye = [ 0.2, yb[1]];

    src_joint.data = {
        x_thigh: [-0.4, 0], y_thigh: [0,0],
        x_shank: xb, y_shank: yb,
        x_flex: xf, y_flex: yf,
        x_ext:  xe, y_ext:  ye
    };

    // Line width ~ force
    flex_line.glyph.line_width = 2 + 6*(F_flexor[i]  / Math.max(1e-9, FmaxF));
    ext_line.glyph.line_width  = 2 + 6*(F_extensor[i]/ Math.max(1e-9, FmaxE));

    // Sync cursors, IK dot & angle label (deg)
    cursor1.location = t[i]; cursor2.location = t[i]; cursor3.location = t[i];
    angle_label.text = "Angle: " + theta_deg[i].toFixed(1) + "°";
    src_marker.data = {x: [t[i]], y: [theta_deg[i]]};

    src_forces.change.emit(); src_torque.change.emit(); src_joint.change.emit();
""")

sFmaxF.js_on_change("value", callback)
sFmaxE.js_on_change("value", callback)
sTime.js_on_change("value", callback)

# ---------------- Play ----------------
play_cb = CustomJS(args=dict(sTime=sTime, N=len(t)), code="""
    var i = sTime.value;
    function step() { i = (i+1) % N; sTime.value = i; if (window._play) setTimeout(step, 50); }
    if (!window._play) { window._play = true; step(); } else { window._play = false; }
""")
btnPlay.js_on_event("button_click", play_cb)

# ---------------- Layout ----------------
grid = gridplot([[p1, p2],[p3, p4]])
layout = column(grid, row(sFmaxF, sFmaxE, sTime, btnPlay))
show(layout)

